# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

# ## elastic net
# EQTL_MODEL = "ELASTIC_NET"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 10
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: GTEX_V8"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/gene_corrs-chr10.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite/gene_corrs-chr10.pkl"

[1] "Output file exists, it will be overwritten"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  eigenvalues <- eigen(data)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  if (length(nonpositive_eigenvalues) > 0) {
    IRdisplay::display("We need to correct the data and make the matrix positive definite")
    return(FALSE)
  } else {
    IRdisplay::display("Matrix is already positive definite!")
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 245 245

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000261456,ENSG00000148377,ENSG00000067064,ENSG00000067057,ENSG00000067082,ENSG00000187134,ENSG00000151632,ENSG00000196139,ENSG00000173848,ENSG00000196372
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000261456,1.000000000,-0.1228503,0.050842088,0.003262422,0.09813451,-0.07818829,0.03809427,-0.005629585,0.00000000,0.0000000000
ENSG00000148377,-0.122850300,1.0000000,-0.170758739,-0.125378774,-0.22860343,0.46265704,0.24080450,0.211018493,-0.16858282,0.1502695159
ENSG00000067064,0.050842088,-0.1707587,1.000000000,0.002722405,-0.02499134,0.17241602,-0.02510686,0.021679597,-0.01466676,-0.0163451020
ENSG00000067057,0.003262422,-0.1253788,0.002722405,1.000000000,-0.07886797,0.05611301,-0.05881128,0.060549470,0.12112833,0.0003274758
ENSG00000067082,0.098134513,-0.2286034,-0.024991338,-0.078867973,1.00000000,0.11128953,-0.05788965,0.038038686,-0.03006517,-0.0053528156
ENSG00000187134,-0.078188288,0.4626570,0.172416017,0.056113009,0.11128953,1.00000000,-0.14740737,0.044083754,0.09026946,0.0107116897


# Check positive definiteness

In [17]:
is_positive_definite(gene_corrs)

[1] "We need to correct the data and make the matrix positive definite"

[1] FALSE

In [18]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs)$values

In [19]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 12

[1] -0.007262097 -0.023304503 -0.058715778 -0.081362165 -0.098671056
 [6] -0.201103749 -0.271518324 -0.292560039 -0.418914817 -0.486092548
[11] -0.654411089 -1.432475918

In [20]:
if (length(eigenvalues[eigenvalues <= 0]) == 0) { quit() }

# Make matrix positive definite if needed

In [21]:
gene_corrs_corrected <- CorrectCM(gene_corrs, 1)

In [22]:
dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)

In [23]:
dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)

In [24]:
gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

In [25]:
dim(gene_corrs_corrected)

[1] 245 245

# Check positive definiteness of corrected matrix

In [26]:
is_positive_definite(gene_corrs_corrected)

[1] "Matrix is already positive definite!"

[1] TRUE

In [27]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs_corrected)$values

In [28]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 0

numeric(0)

In [29]:
stopifnot(length(eigenvalues[eigenvalues <= 0]) == 0)

In [30]:
# quick and visual comparison of the two matrices
IRdisplay::display(head(gene_corrs[1:10, 1:10]))
IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))

,ENSG00000261456,ENSG00000148377,ENSG00000067064,ENSG00000067057,ENSG00000067082,ENSG00000187134,ENSG00000151632,ENSG00000196139,ENSG00000173848,ENSG00000196372
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000261456,1.000000000,-0.1228503,0.050842088,0.003262422,0.09813451,-0.07818829,0.03809427,-0.005629585,0.00000000,0.0000000000
ENSG00000148377,-0.122850300,1.0000000,-0.170758739,-0.125378774,-0.22860343,0.46265704,0.24080450,0.211018493,-0.16858282,0.1502695159
ENSG00000067064,0.050842088,-0.1707587,1.000000000,0.002722405,-0.02499134,0.17241602,-0.02510686,0.021679597,-0.01466676,-0.0163451020
ENSG00000067057,0.003262422,-0.1253788,0.002722405,1.000000000,-0.07886797,0.05611301,-0.05881128,0.060549470,0.12112833,0.0003274758
ENSG00000067082,0.098134513,-0.2286034,-0.024991338,-0.078867973,1.00000000,0.11128953,-0.05788965,0.038038686,-0.03006517,-0.0053528156
ENSG00000187134,-0.078188288,0.4626570,0.172416017,0.056113009,0.11128953,1.00000000,-0.14740737,0.044083754,0.09026946,0.0107116897


,ENSG00000261456,ENSG00000148377,ENSG00000067064,ENSG00000067057,ENSG00000067082,ENSG00000187134,ENSG00000151632,ENSG00000196139,ENSG00000173848,ENSG00000196372
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000261456,1.000000000,-0.07866998,0.02441743,-0.005833793,0.04965977,-0.05152103,-0.0007411962,-0.015379821,0.01062930,-0.012226972
ENSG00000148377,-0.078669984,1.00000000,-0.04270427,-0.049107346,-0.06236423,0.16954034,0.0683660520,0.096484323,-0.05573757,0.048874932
ENSG00000067064,0.024417434,-0.04270427,1.00000000,0.028865861,0.02189621,0.03003104,-0.0331033268,0.006615143,0.02189325,-0.046144297
ENSG00000067057,-0.005833793,-0.04910735,0.02886586,1.000000000,-0.02241157,0.00139159,-0.0345122899,0.010548719,0.08533461,0.007425705
ENSG00000067082,0.049659773,-0.06236423,0.02189621,-0.022411573,1.00000000,0.01598921,-0.0690744346,-0.004384348,0.01997228,-0.018717249
ENSG00000187134,-0.051521033,0.16954034,0.03003104,0.001391590,0.01598921,1.00000000,-0.0526588272,0.060574818,0.01474868,0.041970326


Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).

# Save

In [31]:
py_save_object(gene_corrs_corrected, OUTPUT_FILE)